<a href="https://colab.research.google.com/github/harshitadd/Celestini/blob/master/GeneralisedPredictionHDR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [96]:
!pip install firebase-admin
!pip install joblib
from sklearn.externals import joblib
import firebase_admin
from firebase_admin import credentials, db , storage
import numpy as np 
import cv2
import math 
import pandas as pd
from google.colab.patches import cv2_imshow


/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [0]:
cred = credentials.Certificate("credentials.json")

# Initialize the app with a service account, granting admin privileges
app = firebase_admin.initialize_app(cred, {
    'storageBucket': 'fir-4ca2c.appspot.com',
}, name='storage')

In [0]:
s1=joblib.load('S1.pkl')
s2=joblib.load('S2.pkl')
s3=joblib.load('S3.pkl')
s4=joblib.load('S4.pkl')
s5=joblib.load('S5.pkl')
s6=joblib.load('S6.pkl')
s7=joblib.load('S7.pkl')
s8=joblib.load('S8.pkl')
s9=joblib.load('S9.pkl')
s10=joblib.load('S10.pkl')
s11=joblib.load('S11.pkl')

reg = joblib.load('AQImodel.pkl')
lineareg = joblib.load('AQImodelLinear.pkl')

In [73]:
##EXPERIA 

bucket = storage.bucket(app=app)
blobs = bucket.list_blobs()

#blobs is an interator :google.api_core.page_iterator.HTTPIterator object at 0x7f2cd11c84e0>

imgEXPERIA = [] #has all the lenovo images by their name 
ctr=0
mits = set()
for blob in blobs:
  if blob.name[7:26]=="predict/bhrigu/hdr/":
    blob.download_to_filename('/content/' + "EXPERIA" + str(blob.name.split('/')[-1])+ '.jpeg')
    imgEXPERIA.append("EXPERIA" + str(blob.name.split('/')[-1])+ '.jpeg') 
    print("EXPERIA" + str(blob.name.split('/')[-1])+ '.jpeg')
  if blob.name[7:26]=="predict/xperia/hdr/":
    print('reached')
    blob.download_to_filename('/content/' + "EXPERIA" + str(blob.name.split('/')[-1])+ '.jpeg')
    imgEXPERIA.append("EXPERIA" + str(blob.name.split('/')[-1])+ '.jpeg') 
    print("EXPERIA" + str(blob.name.split('/')[-1])+ '.jpeg')
      

EXPERIA2019-07-22 15:29:19.jpeg
EXPERIA2019-07-22 15:34:19.jpeg
EXPERIA2019-07-22 15:39:19.jpeg
EXPERIA2019-07-22 15:44:19.jpeg
EXPERIA2019-07-22 15:49:19.jpeg
EXPERIA2019-07-22 15:54:19.jpeg
EXPERIA2019-07-22 16:00:21.jpeg
EXPERIA2019-07-22 16:04:19.jpeg
EXPERIA2019-07-22 16:09:19.jpeg
EXPERIA2019-07-22 16:14:19.jpeg
EXPERIA2019-07-22 16:20:21.jpeg
EXPERIA2019-07-22 16:24:19.jpeg
EXPERIA2019-07-22 16:29:19.jpeg
EXPERIA2019-07-22 16:34:19.jpeg
EXPERIA2019-07-22 16:39:19.jpeg
EXPERIA2019-07-22 16:44:19.jpeg
EXPERIA2019-07-22 16:49:19.jpeg
EXPERIA2019-07-22 16:54:19.jpeg
EXPERIA2019-07-22 16:59:19.jpeg
EXPERIA2019-07-22 17:04:19.jpeg
EXPERIA2019-07-22 17:09:19.jpeg
EXPERIA2019-07-22 17:14:19.jpeg
EXPERIA2019-07-22 17:19:19.jpeg
EXPERIA2019-07-22 17:24:19.jpeg
EXPERIA2019-07-22 17:29:19.jpeg
EXPERIA2019-07-22 17:34:19.jpeg
EXPERIA2019-07-22 17:36:01.jpeg
EXPERIA2019-07-22 17:37:32.jpeg
EXPERIA2019-07-22 17:43:36.jpeg
EXPERIA2019-07-22 17:47:32.jpeg
EXPERIA2019-07-22 17:52:33.jpeg
EXPERIA2

In [74]:
print(len(imgEXPERIA))


41


In [0]:
XHDR =[]

x_trans=[]
x_cont=[]
x_ent=[]

for img in imgEXPERIA:
    im = cv2.imread('/content/' + str(img))
    im = cv2.resize(im, (256,256))
    XHDR.append(cv2.rotate(im, rotateCode = cv2.ROTATE_90_CLOCKWISE))
    
    x_trans.append(transmission(im, retMean = False))
    x_cont.append(contrast(im))
    x_ent.append(entropy(im))

In [0]:
cred = credentials.Certificate('credentials.json')

# Initialize the app with a service account, granting admin privileges
firebase_admin.initialize_app(cred, {
    'databaseURL': 'https://fir-4ca2c.firebaseio.com/'
})

In [0]:
ref = db.reference()
df = ref.get()

In [79]:
temp_weather = {}

for key in df:
  try:
    l=[]
    l.append(df[key]['main']['humidity'])
    l.append(df[key]['main']['pressure'])
    l.append(df[key]['main']['temp'])
    l.append(df[key]['visibility'])
    l.append(df[key]['wind']['deg'])
    l.append(df[key]['wind']['speed'])
    l.append(df[key]['clouds']['all'])
    k = df[key]['weather'][0]['description']
    if(k=='haze'):
      l.append(0)
    elif(k=='mist'):
      l.append(1)
    elif(k=='drizzle'):
      l.append(2)
    elif(k=='dust'):
      l.append(3)
    temp_weather[key]=l
  except:
    print(key)
 

08-07-2019 02:52:50
08-07-2019 02:57:50
13-07-2019 12:43:49


In [0]:
print(len(temp_weather))

In [0]:
features=[]
for key in temp_weather:
  i=0
  for k in imgEXPERIA:
    i+=1
    temp = k[12:-5]
    temp = temp.split('-')
    newk = temp[1].split(' ')
    final = newk[0] + '-' + temp[0] + '-' + '2019' + ' ' + newk[1]
    if(final[:-4] == key[:-4]):
      try:
        l = []
        l.append(key[11:13])
        l.append(x_trans[i])
        l.append(x_ent[i])
        l.append(x_cont[i])
        l.append(temp_weather[key])
        features.append(l)
        break
      except:
        pass


In [89]:
print(features[0])

['15', array([[0.        , 0.        , 0.        , ..., 0.50588235, 0.50980392,
        0.50980392],
       [0.        , 0.        , 0.        , ..., 0.50588235, 0.51372549,
        0.51372549],
       [0.        , 0.        , 0.        , ..., 0.50588235, 0.51372549,
        0.51372549],
       ...,
       [0.        , 0.        , 0.        , ..., 0.50196078, 0.50588235,
        0.50588235],
       [0.        , 0.        , 0.        , ..., 0.50196078, 0.50588235,
        0.50588235],
       [0.        , 0.        , 0.        , ..., 0.50196078, 0.50588235,
        0.50588235]]), 11.699877738952637, 70.39109838990026, [56, 1000, 34.58, 3000, 90, 1, 75, 0]]


In [0]:
hours=[]
hum=[]
pressure=[]
temp=[]
vis=[]
deg=[]
speed=[]
cc=[]
desc=[]
trans=[]
cont=[]
ent=[]
pm=[]


for e in features:
  #print(e[0])
  hours.append(float(e[0]))
  trans.append(e[1])
  ent.append(float(e[2]))
  cont.append(float(e[3]))
  hum.append(e[4][0])
  pressure.append(e[4][1])
  temp.append(e[4][2]) 
  vis.append(e[4][3])
  deg.append(e[4][4])
  speed.append(e[4][5])
  cc.append(e[4][6])
  try:
    desc.append(e[4][7])
  except:
    #print(e[0])
    desc.append(0)

 

In [0]:
print(len(hours))
print(hours)
print(len(trans))
print(trans)
print(len(cont))

print(speed)
print(len(cc))
print(cc)
print(len(desc))
print(desc)



In [111]:
print(np.shape(trans[0]))

(256, 256)


In [109]:
final_shape=len(hours)

#FEATURES 


trans = np.reshape(trans,(final_shape,1))

hours = np.reshape(hours,(final_shape,1))
cont = np.reshape(cont,(final_shape,1))
ent = np.reshape(ent,(final_shape,1))
hum = np.reshape(hum,(final_shape,1))
pressure = np.reshape(pressure,(final_shape,1))
temp = np.reshape(temp,(final_shape,1))
vis = np.reshape(vis,(final_shape,1))
deg = np.reshape(deg,(final_shape,1))
speed = np.reshape(speed,(final_shape,1))
cc = np.reshape(cc,(final_shape,1))
desc = np.reshape(desc,(final_shape,1))


hours = s1.fit_transform(hours)
cont = s2.fit_transform(cont)
ent = s3.fit_transform(ent)
hum = s4.fit_transform(hum)
pressure = s5.fit_transform(pressure)
temp = s6.fit_transform(temp)
vis = s7.fit_transform(vis)
deg = s8.fit_transform(deg)
speed = s9.fit_transform(speed)
cc = s10.fit_transform(cc)
des = s11.fit_transform(desc)


test = np.hstack((hours,cont, ent, hum, pressure, temp, vis, deg, speed, cc, desc,trans))



ValueError: ignored

In [106]:
print(len(test[0]))

11


In [105]:

print(reg.predict(test))



ValueError: ignored

In [0]:
bucket = storage.bucket(app=app)
blobs = bucket.list_blobs()

#blobs is an interator :google.api_core.page_iterator.HTTPIterator object at 0x7f2cd11c84e0>

imgS7HDR = [] #has all the lenovo images by their name 
ctr=0
mits = set()
for blob in blobs:
  if blob.name[7:21]=="predict/s7/hdr":
    blob.download_to_filename('/content/' + "S7HDR" + str(blob.name.split('/')[-1])+ '.jpeg')
    imgS7HDR.append("S7HDR" + str(blob.name.split('/')[-1])+ '.jpeg') 
    print("S7HDR" + str(blob.name.split('/')[-1])+ '.jpeg')
      
  
     

In [0]:
bucket = storage.bucket(app=app)
blobs = bucket.list_blobs()

#blobs is an interator :google.api_core.page_iterator.HTTPIterator object at 0x7f2cd11c84e0>

imgASUSHDR = [] #has all the lenovo images by their name 
ctr=0
mits = set()
for blob in blobs:
  if blob.name[7:28]=="predict/divyanshu/hdr":
    blob.download_to_filename('/content/' + "DIVHDR" + str(blob.name.split('/')[-1])+ '.jpeg')
    imgASUSHDR.append("DIVHDR" + str(blob.name.split('/')[-1])+ '.jpeg') 
    print("DIVHDR" + str(blob.name.split('/')[-1])+ '.jpeg')
      

In [0]:
bucket = storage.bucket(app=app)
blobs = bucket.list_blobs()

#blobs is an interator :google.api_core.page_iterator.HTTPIterator object at 0x7f2cd11c84e0>

imgMI = [] #has all the lenovo images by their name 
ctr=0
mits = set()
for blob in blobs:
  if blob.name[7:25]=="predict/minote/hdr":
    blob.download_to_filename('/content/' + "MIHDR" + str(blob.name.split('/')[-1])+ '.jpeg')
    imgMI.append("MIHDR" + str(blob.name.split('/')[-1])+ '.jpeg') 
    print("MIHDR" + str(blob.name.split('/')[-1])+ '.jpeg')

In [0]:
print(len(imgMI))

In [0]:
MIHDR =[] 

mi_trans=[]
mi_cont=[]
mi_ent=[]

for img in imgMI:
  im = cv2.imread('/content/'+str(img))
  im = cv2.resize(im, (256,256))
  MIHDR.append(cv2.rotate(im, rotateCode = cv2.ROTATE_90_CLOCKWISE))
  mi_trans.append(transmission(im,retMean=False))
  mi_cont.append(contrast(im))
  mi_ent.append(entropy)

In [0]:
print(len(x_ent))

In [0]:
s7_trans=[]
s7_cont=[]
s7_ent=[]

S7HDR =[]
for img in imgS7HDR:
  im = cv2.imread('/content/' + str(img))
  im = cv2.resize(im, (256,256))
  S7HDR.append(cv2.rotate(im, rotateCode = cv2.ROTATE_90_CLOCKWISE))
  s7_trans.append(transmission(im, retMean = False))
  s7_cont.append(im)
  s7_ent.append(im)


In [0]:
weather={}
for key in match:
  temp=key.split('-')
  newtemp=temp[2].split(' ')
  newesttemp=newtemp[1].split(':')
  print(keytemp)
  keytemp = (temp[0] + '-' + temp[1] + '-' + newtemp[0] + ' ' + newesttemp[0] + ':' + newesttemp[1] )

  j=int(newesttemp[1])
  if(j<=7.5):
    final = temp[0] + '-' + temp[1] + '-' + newtemp[0] + ' ' + newesttemp[0] + ':' + '00'
  elif(j<=15 or j<=22.5):
    final = temp[0] + '-' + temp[1] + '-' + newtemp[0] + ' ' + newesttemp[0] + ':' + '15'
  elif(j<=30 or j<=37.5):
    final = temp[0] + '-' + temp[1] + '-' + newtemp[0] + ' ' + newesttemp[0] + ':' + '30'
  elif(j<=45 or j<=52.5):
    final = temp[0] + '-' + temp[1] + '-' + newtemp[0] + ' ' + newesttemp[0] + ':' + '45'
  elif(j>52.5):
    final = temp[0] + '-' + temp[1] + '-' + newtemp[0] + ' ' + newesttemp[0] + ':' + '00'
  if(final not in weather):
    weather[keytemp]=match[key]
  
 

In [0]:
print(weather)

{'2019-07-22 15:24': ('S7HDR2019-07-22 15:24:46.jpeg', 'DIVHDR2019-07-22 15:26:07.jpeg', 'EXPERIA2019-07-22 15:29:19.jpeg'), '2019-07-22 15:49': ('S7HDR2019-07-22 15:49:50.jpeg', 'DIVHDR2019-07-22 15:46:32.jpeg', 'EXPERIA2019-07-22 15:49:19.jpeg'), '2019-07-22 15:54': ('S7HDR2019-07-22 15:54:50.jpeg', 'DIVHDR2019-07-22 15:55:49.jpeg', 'EXPERIA2019-07-22 15:54:19.jpeg'), '2019-07-22 15:59': ('S7HDR2019-07-22 15:59:50.jpeg', 'DIVHDR2019-07-22 15:55:49.jpeg', 'EXPERIA2019-07-22 15:54:19.jpeg'), '2019-07-22 16:04': ('S7HDR2019-07-22 16:04:51.jpeg', 'DIVHDR2019-07-22 16:05:49.jpeg', 'EXPERIA2019-07-22 16:09:19.jpeg'), '2019-07-22 16:09': ('S7HDR2019-07-22 16:09:51.jpeg', 'DIVHDR2019-07-22 16:05:49.jpeg', 'EXPERIA2019-07-22 16:09:19.jpeg'), '2019-07-22 16:14': ('S7HDR2019-07-22 16:14:50.jpeg', 'DIVHDR2019-07-22 16:16:31.jpeg', 'EXPERIA2019-07-22 16:14:19.jpeg'), '2019-07-22 16:19': ('S7HDR2019-07-22 16:19:50.jpeg', 'DIVHDR2019-07-22 16:16:31.jpeg', 'EXPERIA2019-07-22 16:14:19.jpeg'), '2019-0

In [0]:
S7_trans=[]
S7_cont=[]
S7_ent=[]


for key in weather:  
  ## S7 
  img = weather[key][0]
  im = cv2.imread('/content/' + str(img))
  im = cv2.resize(im, (256,256))
  ASUSHDR.append(cv2.rotate(im, rotateCode = cv2.ROTATE_90_CLOCKWISE))
  S7_trans.append(transmission(im, retMean = False))
  S7_cont.append(contrast(im))
  S7_ent.append(entropy(im))


In [0]:
print(len(S7_ent))

20


In [0]:

print(len(temp_weather))

In [0]:
print(temp_weather)

In [0]:
print(temp_weather)
print(weather)

In [0]:

for key in temp_weather:
  print(key[:-1])
  for w in weather:
    print(w[:-1])
    if(key[:-4]==w[:-2]):
      print(key)
    break

In [0]:
hours=[]
hum=[]
pressure=[]
temp=[]
vis=[]
deg=[]
speed=[]
cc=[]
desc=[]
trans=[]
cont=[]
ent=[]
pm=[]


for e in feat:
  #print(e[0])
  hours.append(float(e[0][-5:-3]))
  pm.append(float(e[4]))
  trans.append(e[1])
  ent.append(float(e[2]))
  cont.append(float(e[3]))
  for w in weather:                                      ##Matching weather data 
    if(e[0] == w):
      hum.append(weather[w][0])
      pressure.append(weather[w][1])
      temp.append(weather[w][2]) 
      vis.append(weather[w][3])
      deg.append(weather[w][4])
      speed.append(weather[w][5])
      cc.append(weather[w][6])
      try:
        desc.append(weather[w][7])
      except:
        #print(e[0])
        desc.append(0)
    

In [0]:
def transmission(img, lt = 230, retMean = False):
  ker = np.ones((3, 3))/9.0
  imgrgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  b, g, r = cv2.split(img)
  hsv = cv2.cvtColor(imgrgb, cv2.COLOR_RGB2HSV)
  _, _, v = cv2.split(hsv)
  v_blur = cv2.filter2D(v, -1, ker)
  _,building_mask = cv2.threshold(v_blur, lt, 255, cv2.THRESH_BINARY)
  sky = cv2.bitwise_and(imgrgb, imgrgb, mask = building_mask)
  sky_mask = cv2.bitwise_not(building_mask, building_mask.copy())
  building = cv2.bitwise_and(imgrgb, imgrgb, mask = sky_mask)
 
  al = max(v_blur.flatten())
  #Airlight Found. Now calculate Transmission Map
  img_norm = img/float(al)
  b, g, r = cv2.split(img_norm)
  kernel = np.ones((3,3),np.uint8)
  im_new = np.zeros((img.shape[0], img.shape[1]))
  for i in range(img.shape[0]):
    for j in range(img.shape[1]):
      im_new[i][j] = min(b[i][j], g[i][j], r[i][j])
  dcp = cv2.erode(im_new, kernel, iterations = 1)
  #ax[3].imshow(im_new, cmap = 'gray')
  #ax[4].imshow(dcp, cmap = 'gray')
  tr = np.ones((dcp.shape[0], dcp.shape[1]))
  for i in range(dcp.shape[0]):
    for j in range(dcp.shape[1]):
      tr[i][j] = 1 - dcp[i][j]
  #ax[1].imshow(tr, cmap = 'gray')
  if retMean == False:
    return tr
  elif retMean == True:
    return np.mean(tr)

In [0]:
def entropy(image):
  image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  hist = cv2.calcHist( [image.astype('float32')],
              [0], 
              None,
               [256], 
              [0,256] )
  h_norm = 0
  h_norm = cv2.normalize(hist, h_norm)
  ent = 0
  for p in h_norm:
    try:
      ent += p*math.log(p, 2)
    except:
      pass
    
  return (-1 * ent[0])

In [0]:
def contrast(image):
  image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  s = 0
  ss = 0
  mean = np.mean(image)
  x = (-1 * mean * (np.ones(image.shape)))
  ss = np.square(np.add(image, x))
  cont = np.sqrt(np.sum(ss)/float(image.size))
  return cont